In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import cv2
import numpy as np
from skimage.exposure import match_histograms

from skimage.segmentation import slic, mark_boundaries

dataset_dir = '/kaggle/input/remote-sensing-satellite-images/Remote Sensing Data.v2i.yolov8/train/images'
processed_dir = '/kaggle/working/processed_images'
os.makedirs(processed_dir, exist_ok=True)

def is_outlier(image):
    try:
        mean_value = np.mean(image)
        return mean_value < 30 or mean_value > 220
    except:
        return False

def noise_reduction(image):
    try:
        median_filtered = cv2.medianBlur(image, 3)
        gaussian_filtered = cv2.GaussianBlur(median_filtered, (3, 3), 0)
        return gaussian_filtered
    except:
        return image

def resize_image(image, target_size=(256, 256)):
    try:
        h, w = image.shape[:2]
        scale = min(target_size[0] / h, target_size[1] / w)
        resized = cv2.resize(image, (int(w * scale), int(h * scale)), interpolation=cv2.INTER_AREA)
        delta_w = target_size[1] - resized.shape[1]
        delta_h = target_size[0] - resized.shape[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        return cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    except:
        return image

def adaptive_hist_equalization(image):
    try:
        lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
        l, a, b = cv2.split(lab_image)
        l_adaptive = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(l)
        lab_image_adaptive = cv2.merge((l_adaptive, a, b))
        return cv2.cvtColor(lab_image_adaptive, cv2.COLOR_Lab2BGR)
    except:
        return image

def gamma_correction(image, gamma=1.5):
    try:
        inv_gamma = 1.0 / gamma
        table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
        return cv2.LUT(image, table)
    except:
        return image

def add_rain(image):
    try:
        rain_streaks = np.zeros_like(image, dtype=np.uint8)
        count = 1000
        for _ in range(count):
            x1, y1 = np.random.randint(0, image.shape[1]), np.random.randint(0, image.shape[0])
            x2, y2 = x1 + np.random.randint(-5, 5), y1 + np.random.randint(10, 20)
            cv2.line(rain_streaks, (x1, y1), (x2, y2), (255, 255, 255), 1)
        return cv2.addWeighted(image, 0.8, rain_streaks, 0.2, 0)
    except:
        return image

def add_fog(image):
    try:
        fog = np.zeros_like(image, dtype=np.uint8)
        cv2.circle(fog, (image.shape[1] // 2, image.shape[0] // 2), max(image.shape[:2]), (255, 255, 255), -1)
        return cv2.addWeighted(image, 0.7, fog, 0.3, 0)
    except:
        return image

def add_snow(image):
    try:
        snow = np.zeros_like(image, dtype=np.uint8)
        num_snowflakes = 1000
        for _ in range(num_snowflakes):
            x, y = np.random.randint(0, image.shape[1]), np.random.randint(0, image.shape[0])
            snow[y, x] = (255, 255, 255)
        return cv2.addWeighted(image, 0.8, snow, 0.2, 0)
    except:
        return image

def add_rectangular_occlusion(image):
    try:
        x1, y1 = np.random.randint(0, image.shape[1] // 2), np.random.randint(0, image.shape[0] // 2)
        x2, y2 = x1 + np.random.randint(20, 50), y1 + np.random.randint(20, 50)
        return cv2.rectangle(image.copy(), (x1, y1), (x2, y2), (0, 0, 0), -1)
    except:
        return image

def add_circular_occlusion(image):
    try:
        x, y = np.random.randint(0, image.shape[1]), np.random.randint(0, image.shape[0])
        radius = np.random.randint(10, 30)
        return cv2.circle(image.copy(), (x, y), radius, (0, 0, 0), -1)
    except:
        return image


def histogram_matching(image, reference_image):
    try:
        matched = match_histograms(image, reference_image, multichannel=True)
        return matched
    except:
        return image

def edge_enhancement(image):
    try:
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        edges = cv2.Laplacian(gray_image, cv2.CV_64F)
        edges = cv2.convertScaleAbs(edges)
        enhanced = cv2.addWeighted(image, 0.8, cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB), 0.2, 0)
        return enhanced
    except:
        return image
def cloud_removal(image):
    try:
        hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        _, mask = cv2.threshold(hsv_image[:, :, 2], 200, 255, cv2.THRESH_BINARY)
        cleaned = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)
        return cleaned
    except:
        return image
def calculate_ndvi(image):
    try:
        b, g, r = cv2.split(image)
        ndvi = (r.astype(float) - b.astype(float)) / (r.astype(float) + b.astype(float) + 1e-6)
        ndvi_normalized = ((ndvi + 1) / 2 * 255).astype(np.uint8)
        return cv2.applyColorMap(ndvi_normalized, cv2.COLORMAP_JET)
    except:
        return image

def superpixel_segmentation(image):
    try:
        segments = slic(image, n_segments=100, compactness=10, start_label=1)
        segmented_image = mark_boundaries(image, segments, color=(255, 0, 0))
        return (segmented_image * 255).astype(np.uint8)
    except:
        return image
def color_normalization(image):
    try:
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
        l, a, b = cv2.split(lab)
        l_norm = cv2.equalizeHist(l)
        lab_norm = cv2.merge((l_norm, a, b))
        return cv2.cvtColor(lab_norm, cv2.COLOR_Lab2RGB)
    except:
        return image
def shadow_removal(image):
    try:
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        shadow_mask = hsv[:, :, 2] < 50
        shadow_corrected = image.copy()
        shadow_corrected[shadow_mask] = cv2.equalizeHist(image[shadow_mask])
        return shadow_corrected
    except:
        return image
def overlay_boundaries(image, annotations):
    try:
        for annotation in annotations:
            cv2.rectangle(image, annotation[0], annotation[1], (255, 0, 0), 2)
        return image
    except:
        return image


image_files = [f for f in os.listdir(dataset_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
processed_count = 0

for i, image_file in enumerate(image_files):
    image_path = os.path.join(dataset_dir, image_file)
    img = cv2.imread(image_path)
    
    if img is None:
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if is_outlier(img):
        continue

    try:
        img_processed = noise_reduction(img)
        img_processed = resize_image(img_processed)
        img_processed = adaptive_hist_equalization(img_processed)
        img_processed = gamma_correction(img_processed)
        img_processed = add_rain(img_processed)
        img_processed = add_fog(img_processed)
        img_processed = add_snow(img_processed)
        img_processed = add_rectangular_occlusion(img_processed)
        img_processed = add_circular_occlusion(img_processed)
        img_processed = calculate_ndvi(img_processed)
        img_processed = edge_enhancement(img_processed)
        img_processed = superpixel_segmentation(img_processed)
        img_processed = color_normalization(img_processed)
        img_processed = shadow_removal(img_processed)
        img_processed = overlay_boundaries(img_processed)
        
        
        
    except:
        img_processed = img  

    processed_image_path = os.path.join(processed_dir, f'processed_{i + 1:04d}.png')
    cv2.imwrite(processed_image_path, cv2.cvtColor(img_processed, cv2.COLOR_RGB2BGR))
    processed_count += 1

print(f"Processing complete. {processed_count} images were successfully processed.")


Processing complete. 700 images were successfully processed.


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

processed_dir = '/kaggle/working/processed_images'

image_files = sorted([f for f in os.listdir(processed_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

# Display the first 5 images
plt.figure(figsize=(15, 10))
for i, image_file in enumerate(image_files[:5]):
    image_path = os.path.join(processed_dir, image_file)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for correct visualization
    
    plt.subplot(1, 5, i + 1)
    plt.imshow(img)
    plt.title(f"Image {i+1}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Directories for original and processed images
original_dir = '/kaggle/input/remote-sensing-satellite-images/Remote Sensing Data.v2i.yolov8/train/images'
processed_dir = '/kaggle/working/processed_images'

# Get a sorted list of image files
original_files = sorted([f for f in os.listdir(original_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
processed_files = sorted([f for f in os.listdir(processed_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

# Display the first 5 images side by side
plt.figure(figsize=(15, 10))
for i in range(5):
    # Original image
    original_path = os.path.join(original_dir, original_files[i])
    original_img = cv2.imread(original_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    
    # Processed image
    processed_path = os.path.join(processed_dir, processed_files[i])
    processed_img = cv2.imread(processed_path)
    processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)
    
    # Display original image
    plt.subplot(5, 2, 2 * i + 1)
    plt.imshow(original_img)
    plt.title(f"Original {i+1}")
    plt.axis('off')
    
    # Display processed image
    plt.subplot(5, 2, 2 * i + 2)
    plt.imshow(processed_img)
    plt.title(f"Processed {i+1}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt

# Function to visualize the original and processed images side by side
def visualize_processing(original, processed, step_name):
    try:
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(original)
        plt.title("Original Image")
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.imshow(processed)
        plt.title(f"After {step_name}")
        plt.axis("off")

        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Error visualizing step {step_name}: {e}")

image_files = [f for f in os.listdir(dataset_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
processed_count = 0

for i, image_file in enumerate(image_files):
    image_path = os.path.join(dataset_dir, image_file)
    img = cv2.imread(image_path)
    
    if img is None:
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if is_outlier(img):
        continue

    try:
        img_processed = img
        steps = [
            (noise_reduction, "Noise Reduction"),
            (resize_image, "Resizing"),
            (adaptive_hist_equalization, "Adaptive Histogram Equalization"),
            (gamma_correction, "Gamma Correction"),
            (add_rain, "Add Rain Effect"),
            (add_fog, "Add Fog Effect"),
            (add_snow, "Add Snow Effect"),
            (add_rectangular_occlusion, "Rectangular Occlusion"),
            (add_circular_occlusion, "Circular Occlusion"),
        
            (edge_enhancement, "Edge Enhancement"),
            (superpixel_segmentation, "Superpixel Segmentation"),
            (color_normalization, "Color Normalization"),
            (shadow_removal, "Shadow Removal"),
        ]

        for step_function, step_name in steps:
            prev_img = img_processed
            img_processed = step_function(img_processed)
            visualize_processing(prev_img, img_processed, step_name)

    except Exception as e:
        print(f"Error processing image {image_file}: {e}")
        img_processed = img

    processed_image_path = os.path.join(processed_dir, f'processed_{i + 1:04d}.png')
    cv2.imwrite(processed_image_path, cv2.cvtColor(img_processed, cv2.COLOR_RGB2BGR))
    processed_count += 1

print(f"Processing complete. {processed_count} images were successfully processed.")
